In [2]:
import pandas as pd

# Load recent performance data
batting_df = pd.read_csv("/content/batting_stats25.csv")
bowling_df = pd.read_csv("/content/bowlingstats25.csv")

# Clean names (if needed) by stripping spaces and ensuring consistency
batting_df['Player'] = batting_df['Player'].str.strip()
bowling_df['Player'] = bowling_df['Player'].str.strip()

# Select relevant columns
batting_stats = batting_df[['Player', 'Runs', 'Avg', 'SR', '4s', '6s']]
bowling_stats = bowling_df[['Player', 'Wkts', 'Avg', 'Econ', 'SR', '4w', '5w']]

# Rename columns to distinguish
batting_stats = batting_stats.rename(columns={
    'Avg': 'Recent_Batting_Avg',
    'SR': 'Recent_Batting_SR',
    '4s': 'Recent_4s',
    '6s': 'Recent_6s'
})

bowling_stats = bowling_stats.rename(columns={
    'Avg': 'Recent_Bowling_Avg',
    'Econ': 'Recent_Economy_Rate',
    'SR': 'Recent_Bowling_SR',
    '4w': 'Recent_4w',
    '5w': 'Recent_5w'
})

# Merge the two based on player name (outer join to preserve all players)
recent_stats = pd.merge(batting_stats, bowling_stats, on='Player', how='outer')

# Display merged stats
print(recent_stats.head())
recent_stats.to_csv("recent_player_stats.csv", index=False)

            Player   Runs  Recent_Batting_Avg  Recent_Batting_SR  Recent_4s  \
0  Abhishek Sharma   91.0               30.33             178.43        8.0   
1       Adam Zampa   32.0               16.00             228.57        1.0   
2   Ajinkya Rahane  123.0               30.75             153.75        9.0   
3     Akeal Hosein   10.0                5.00            1000.00        1.0   
4   Alzarri Joseph   20.0               10.00             666.67        1.0   

   Recent_6s  Wkts  Recent_Bowling_Avg  Recent_Economy_Rate  \
0        6.0   NaN                 NaN                  NaN   
1        1.0   2.0                47.0                11.75   
2       10.0   NaN                 NaN                  NaN   
3        1.0   NaN                 NaN                  NaN   
4        1.0   NaN                 NaN                  NaN   

   Recent_Bowling_SR  Recent_4w  Recent_5w  
0                NaN        NaN        NaN  
1               24.0        0.0        0.0  
2          

In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Load recent performance data
batting_df = pd.read_csv("/content/batting_stats25.csv")
bowling_df = pd.read_csv("/content/bowlingstats25.csv")

# Clean names
batting_df['Player'] = batting_df['Player'].str.strip()
bowling_df['Player'] = bowling_df['Player'].str.strip()

# Select and rename relevant columns
batting_stats = batting_df[['Player', 'Runs', 'Avg', 'SR', '4s', '6s']].rename(columns={
    'Avg': 'Recent_Batting_Avg',
    'SR': 'Recent_Batting_SR',
    '4s': 'Recent_4s',
    '6s': 'Recent_6s'
})

bowling_stats = bowling_df[['Player', 'Wkts', 'Avg', 'Econ', 'SR', '4w', '5w']].rename(columns={
    'Avg': 'Recent_Bowling_Avg',
    'Econ': 'Recent_Economy_Rate',
    'SR': 'Recent_Bowling_SR',
    '4w': 'Recent_4w',
    '5w': 'Recent_5w'
})

# Merge both stats
recent_stats = pd.merge(batting_stats, bowling_stats, on='Player', how='outer')

# Fill NaN values with 0 (means player didn't bat/bowl)
recent_stats.fillna(0, inplace=True)

# Normalize batting and bowling metrics
scaler = MinMaxScaler()

# Batting metrics to normalize (higher is better)
batting_features = ['Runs', 'Recent_Batting_Avg', 'Recent_Batting_SR', 'Recent_4s', 'Recent_6s']
recent_stats[batting_features] = scaler.fit_transform(recent_stats[batting_features])

# Bowling metrics to normalize
# For bowling: Wkts is good (↑), Econ & Avg & SR are better when lower (↓)
# So we invert Econ, Bowling_Avg, Bowling_SR by multiplying with -1 first
bowling_features = ['Wkts', 'Recent_Bowling_Avg', 'Recent_Economy_Rate', 'Recent_Bowling_SR', 'Recent_4w', 'Recent_5w']

# Invert undesirable stats
recent_stats['Inv_Bowling_Avg'] = -recent_stats['Recent_Bowling_Avg']
recent_stats['Inv_Economy'] = -recent_stats['Recent_Economy_Rate']
recent_stats['Inv_Bowling_SR'] = -recent_stats['Recent_Bowling_SR']

# Normalize
bowling_consistency_features = ['Wkts', 'Inv_Bowling_Avg', 'Inv_Economy', 'Inv_Bowling_SR', 'Recent_4w', 'Recent_5w']
recent_stats[bowling_consistency_features] = scaler.fit_transform(recent_stats[bowling_consistency_features])

# Compute Batting and Bowling Consistency Scores
recent_stats['Batting_Consistency'] = recent_stats[batting_features].mean(axis=1)
recent_stats['Bowling_Consistency'] = recent_stats[bowling_consistency_features].mean(axis=1)

# Final consistency: max of the two
recent_stats['Final_Consistency_Score'] = recent_stats[['Batting_Consistency', 'Bowling_Consistency']].max(axis=1)

# Sort by consistency
recent_stats = recent_stats.sort_values(by='Final_Consistency_Score', ascending=False)

# Save to CSV
recent_stats.to_csv("recent_player_consistency.csv", index=False)

print("✅ Consistency scores calculated and saved to 'recent_player_consistency.csv'")


✅ Consistency scores calculated and saved to 'recent_player_consistency.csv'


In [11]:
import pandas as pd
import numpy as np

# Load datasets
d1 = pd.read_csv("/content/Final_cricket_data.csv")         # Main dataset
batting_df = pd.read_csv("/content/batting_stats25.csv")    # Recent batting stats
bowling_df = pd.read_csv("/content/bowlingstats25.csv")     # Recent bowling stats

# Step 1: Clean player names (remove leading/trailing whitespace)
for df in [d1, batting_df, bowling_df]:
    df['Player_Name'] = df['Player_Name'].str.strip()

# Step 2: Select and rename relevant columns for recent batting and bowling
batting_stats = batting_df[['Player_Name', 'Runs', 'Avg', 'SR', '4s', '6s']].rename(columns={
    'Avg': 'Recent_Batting_Avg',
    'SR': 'Recent_Batting_SR',
    '4s': 'Recent_4s',
    '6s': 'Recent_6s'
})

bowling_stats = bowling_df[['Player_Name', 'Wkts', 'Avg', 'Econ', 'SR', '4w', '5w']].rename(columns={
    'Avg': 'Recent_Bowling_Avg',
    'Econ': 'Recent_Economy_Rate',
    'SR': 'Recent_Bowling_SR',
    '4w': 'Recent_4w',
    '5w': 'Recent_5w'
})

# Step 3: Merge recent stats together
recent_stats = pd.merge(batting_stats, bowling_stats, on='Player_Name', how='outer')

# Step 4: Define positive scoring consistency calculations

def calculate_batting_consistency(row):
    weights = {
        'Runs': 0.3,
        'Recent_Batting_Avg': 0.3,
        'Recent_Batting_SR': 0.2,
        'Recent_4s': 0.1,
        'Recent_6s': 0.1
    }
    score = 0
    for k, w in weights.items():
        score += (row[k] if pd.notnull(row[k]) else 0) * w
    return score

def calculate_bowling_consistency(row):
    score = 0
    if pd.notnull(row['Wkts']):
        score += row['Wkts'] * 0.4
    if pd.notnull(row['Recent_Bowling_Avg']) and row['Recent_Bowling_Avg'] > 0:
        score += (1 / row['Recent_Bowling_Avg']) * 0.2
    if pd.notnull(row['Recent_Economy_Rate']) and row['Recent_Economy_Rate'] > 0:
        score += (1 / row['Recent_Economy_Rate']) * 0.2
    if pd.notnull(row['Recent_Bowling_SR']) and row['Recent_Bowling_SR'] > 0:
        score += (1 / row['Recent_Bowling_SR']) * 0.1
    if pd.notnull(row['Recent_4w']):
        score += row['Recent_4w'] * 0.05
    if pd.notnull(row['Recent_5w']):
        score += row['Recent_5w'] * 0.05
    return score

# Step 5: Apply consistency calculations
recent_stats['Batting_Consistency'] = recent_stats.apply(calculate_batting_consistency, axis=1)
recent_stats['Bowling_Consistency'] = recent_stats.apply(calculate_bowling_consistency, axis=1)

# Step 6: Final consistency score = sum of batting and bowling
recent_stats['Final_Consistency_Score'] = (
    recent_stats['Batting_Consistency'].fillna(0) +
    recent_stats['Bowling_Consistency'].fillna(0)
)

# Step 7: Merge back into main dataset
consistency_scores = recent_stats[['Player_Name', 'Batting_Consistency', 'Bowling_Consistency', 'Final_Consistency_Score']]
d1 = d1.merge(consistency_scores, on='Player_Name', how='left')

# Step 8: Show and/or save final dataset
print(d1[['Player_Name', 'Batting_Consistency', 'Bowling_Consistency', 'Final_Consistency_Score']].head())

# Optional: Save to CSV
d1.to_csv("d1 consistency.csv", index=False)


        Player_Name  Batting_Consistency  Bowling_Consistency  \
0   Andre Siddharth                  NaN                  NaN   
1     Anshul Kamboj                  NaN                  NaN   
2      Deepak Hooda               50.132                  0.0   
3      Devon Conway                  NaN                  NaN   
4  Gurjapneet Singh                  NaN                  NaN   

   Final_Consistency_Score  
0                      NaN  
1                      NaN  
2                   50.132  
3                      NaN  
4                      NaN  


In [ ]:
# prompt: load the  dataset

import pandas as pd
df = pd.read_csv('/content/cricket_data.csv')
df
role_mapping = {
    "Batsman": "BAT",
    "Batter": "BAT",
    "Allrounder": "ALL",
    "All-rounder": "ALL",
    "Wicketkeeper": "WK",
     "WK-Batter":"WK",
    "Bowler": "BOWL"
}

# ✅ Apply role mapping to fix roles
df["Role"] = df["Role"].replace(role_mapping)

In [ ]:
df

,Sno,Player_Name,Role,Team,Auction Price,Year,Matches_Batted,Not_Outs,Runs_Scored,Highest_Score,...,Matches_Bowled,Balls_Bowled,Runs_Conceded,Wickets_Taken,Best_Bowling_Match,Bowling_Average,Economy_Rate,Bowling_Strike_Rate,Four_Wicket_Hauls,Five_Wicket_Hauls
0,1,Ruturaj Gaikwad,BAT,Chennai Super Kings,?18 crore,2024.0,14,3,583.0,108*,...,14,0.0,0,0,0,0,0,0,0,0
1,1,Ruturaj Gaikwad,BAT,Chennai Super Kings,?18 crore,2023.0,16,1,590.0,92,...,16,0.0,0,0,0,0,0,0,0,0
2,1,Ruturaj Gaikwad,BAT,Chennai Super Kings,?18 crore,2022.0,14,0,368.0,99,...,14,0.0,0,0,0,0,0,0,0,0
3,1,Ruturaj Gaikwad,BAT,Chennai Super Kings,?18 crore,2021.0,16,2,635.0,101*,...,16,0.0,0,0,0,0,0,0,0,0
4,1,Ruturaj Gaikwad,BAT,Chennai Super Kings,?18 crore,2020.0,6,2,204.0,72,...,6,0.0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,257,Manoj Bhandage,ALL,Royal Challengers Bengaluru,0,2025.0,30,8,480.0,68*,...,25,480.0,600,20,Mar-30,30,7.5,24,0,0
729,258,Mohit Rathee,BOWL,Royal Challengers Bengaluru,0,2025.0,3,1,8.0,6*,...,18,360.0,440,15,Mar-22,29.3,7.3,24,0,0
730,259,Abhinandan Singh,BAT,Royal Challengers Bengaluru,0,2025.0,7,1,110.0,40,...,0,0.0,0,0,0,0,0,0,0,0
731,260,Zeeshan Ansari,BOWL,Sunrisers Hyderabad,0,2025.0,5,2,15.0,8*,...,20,420.0,500,16,Mar-20,31.3,7.1,26.3,0,0


In [ ]:

# prompt: delete th erows which has unknown in role

df = df[df.Role != 'Unknown']


In [ ]:
df.isna().sum()


,0
Sno,0
Player_Name,0
Role,0
Team,0
Auction Price,0
Year,0
Matches_Batted,0
Not_Outs,0
Runs_Scored,0
Highest_Score,0


In [ ]:
import pandas as pd

# Convert "Year" column to integer (handling NaNs)
df["Year"] = pd.to_numeric(df["Year"], errors='coerce').fillna(0).astype(int)

# Define the relevant columns to calculate the average
columns_to_avg = [
    "Matches_Batted", "Not_Outs", "Runs_Scored", "Highest_Score", "Batting_Average", "Balls_Faced",
    "Batting_Strike_Rate", "Centuries", "Half_Centuries", "Fours", "Sixes", "Catches_Taken", "Stumpings",
    "Matches_Bowled", "Balls_Bowled", "Runs_Conceded", "Wickets_Taken", "Best_Bowling_Match", "Bowling_Average",
    "Economy_Rate", "Bowling_Strike_Rate", "Four_Wicket_Hauls", "Five_Wicket_Hauls"
]

# Ensure only numerical columns are considered
df[columns_to_avg] = df[columns_to_avg].apply(pd.to_numeric, errors='coerce')

# Fix: Use a list for multiple groupby columns
df1 = df.groupby(["Player_Name", "Role", "Team"])[columns_to_avg].mean().reset_index()

df1


<ipython-input-5-2fa5ce07cdfa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year"] = pd.to_numeric(df["Year"], errors='coerce').fillna(0).astype(int)
<ipython-input-5-2fa5ce07cdfa>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columns_to_avg] = df[columns_to_avg].apply(pd.to_numeric, errors='coerce')


,Player_Name,Role,Team,Matches_Batted,Not_Outs,Runs_Scored,Highest_Score,Batting_Average,Balls_Faced,Batting_Strike_Rate,...,Matches_Bowled,Balls_Bowled,Runs_Conceded,Wickets_Taken,Best_Bowling_Match,Bowling_Average,Economy_Rate,Bowling_Strike_Rate,Four_Wicket_Hauls,Five_Wicket_Hauls
0,Abdul Samad,ALL,Lucknow Super Giants,8.333333,1.333333,75.333333,21.666667,12.176667,53.000000,118.493333,...,8.333333,18.000000,37.666667,0.666667,NaN,35.000000,10.236667,16.000000,0.000000,0.000000
1,Abdul Samad,BAT,Lucknow Super Giants,12.500000,3.000000,175.500000,NaN,30.225000,118.000000,150.275000,...,12.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
2,Abhinandan Singh,BAT,Royal Challengers Bengaluru,7.000000,1.000000,110.000000,40.000000,18.300000,90.000000,122.200000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
3,Abhinav Manohar,BAT,Sunrisers Hyderabad,6.333333,0.333333,77.000000,31.000000,13.833333,58.000000,112.533333,...,6.333333,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
4,Abhishek Sharma,ALL,Sunrisers Hyderabad,10.000000,0.833333,219.000000,51.500000,19.713333,142.333333,139.790000,...,10.000000,45.000000,64.833333,1.833333,NaN,28.333333,8.853333,19.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Yashasvi Jaiswal,ALL,Rajasthan Royals,10.000000,0.000000,258.000000,68.000000,25.800000,194.000000,132.990000,...,10.000000,1.000000,6.000000,0.000000,NaN,0.000000,36.000000,0.000000,0.000000,0.000000
206,Yashasvi Jaiswal,BAT,Rajasthan Royals,10.500000,0.500000,337.250000,69.333333,29.345000,218.250000,139.657500,...,10.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
207,Yuzvendra Chahal,ALL,Punjab Kings,14.714286,1.714286,5.285714,3.333333,1.904286,11.714286,60.051429,...,14.714286,319.571429,403.714286,19.000000,NaN,22.141429,7.602857,17.698571,0.285714,0.142857
208,Yuzvendra Chahal,BOWL,Punjab Kings,11.400000,0.200000,0.000000,0.000000,0.000000,0.800000,0.000000,...,11.400000,256.800000,355.200000,14.400000,NaN,20.048000,8.300000,14.694000,0.800000,0.000000


In [ ]:
import pandas as pd

# Load datasets
df_players =df1  # Player details, stats (Role, Team, Batting, Bowling, etc.)
df_stats = pd.read_csv("/content/SquadPlayerNames_IndianT20League - SquadData_AllTeams.csv")  # Credits, Player Type, Player_Name, Team

# Standardize column names to ensure consistency
df_players = df_players.rename(columns={'Player Name': 'Player_Name'})
df_stats = df_stats.rename(columns={'Player Name': 'Player_Name'})

# Merge datasets on Player_Name (keeping only matching records)
df_merged = pd.merge(df_players, df_stats[['Player_Name', 'Credits']], on="Player_Name", how="inner")

# Check the first few rows of the merged dataset
print(df_merged)


          Player_Name  Role                         Team  Matches_Batted  \
0         Abdul Samad   ALL         Lucknow Super Giants        8.333333   
1         Abdul Samad   BAT         Lucknow Super Giants       12.500000   
2    Abhinandan Singh   BAT  Royal Challengers Bengaluru        7.000000   
3     Abhinav Manohar   BAT          Sunrisers Hyderabad        6.333333   
4     Abhishek Sharma   ALL          Sunrisers Hyderabad       10.000000   
..                ...   ...                          ...             ...   
188  Yashasvi Jaiswal   ALL             Rajasthan Royals       10.000000   
189  Yashasvi Jaiswal   BAT             Rajasthan Royals       10.500000   
190  Yuzvendra Chahal   ALL                 Punjab Kings       14.714286   
191  Yuzvendra Chahal  BOWL                 Punjab Kings       11.400000   
192    Zeeshan Ansari  BOWL          Sunrisers Hyderabad        5.000000   

     Not_Outs  Runs_Scored  Highest_Score  Batting_Average  Balls_Faced  \
0    1.33333

In [ ]:
# prompt: download the above data intp csv format

df_merged.to_csv('cricket_data_with_credits.csv', index=False)


In [ ]:
# prompt: download

from google.colab import files
files.download('cricket_data_with_credits.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
  # Credits, Player Type, Player_Name, Team

# Standardize column names (strip spaces)
df_players.columns = df_players.columns.str.strip()
df_stats.columns = df_stats.columns.str.strip()

# Merge datasets on 'Player_Name' (keeping 'Credits' from df_stats)
df = pd.merge(df_players, df_stats[['Player_Name', 'Credits']], on="Player_Name", how="inner")

# Define the two teams
team1 = "Gujarat Titans"
team2 = "Punjab Kings"

# Filter players from the selected teams
df_filtered = df[df["Team"].isin([team1, team2])].copy()

# Fill NaN values in credits with a default value (if any missing)
df_filtered["Credits"] = pd.to_numeric(df_filtered["Credits"], errors="coerce").fillna(0)

# Sort players by Credits, Batting_Average, and Bowling_Average
sort_columns = ["Credits"]
if "Batting_Average" in df_filtered.columns:
    sort_columns.append("Batting_Average")
if "Bowling_Average" in df_filtered.columns:
    sort_columns.append("Bowling_Average")

df_filtered = df_filtered.sort_values(by=sort_columns, ascending=[False, False, True])

# Ensure a flexible team selection
selected_team = []
role_count = {"Batsman": 0, "Bowler": 0, "All-Rounder": 0, "Wicketkeeper": 0}
max_credits = 100
current_credits = 0

for _, player in df_filtered.iterrows():
    role = player["Role"]

    # Allow more flexibility in selection
    if current_credits + player["Credits"] > max_credits:
        continue

    if role_count["Wicketkeeper"] < 2:  # Allowing up to 2 wicketkeepers
        selected_team.append(player)
        role_count["Wicketkeeper"] += 1
        current_credits += player["Credits"]
    elif role_count["All-Rounder"] < 3:  # Allowing up to 3 all-rounders
        selected_team.append(player)
        role_count["All-Rounder"] += 1
        current_credits += player["Credits"]
    elif role_count["Batsman"] < 5:  # Allowing up to 5 batsmen
        selected_team.append(player)
        role_count["Batsman"] += 1
        current_credits += player["Credits"]
    elif role_count["Bowler"] < 5:  # Allowing up to 5 bowlers
        selected_team.append(player)
        role_count["Bowler"] += 1
        current_credits += player["Credits"]

    if len(selected_team) == 11:
        break  # Stop when 11 players are selected

# If fewer than 11 players are selected, fill remaining spots with lowest credit players
if len(selected_team) < 11:
    remaining_players = df_filtered[~df_filtered["Player_Name"].isin([p["Player_Name"] for p in selected_team])]
    remaining_players = remaining_players.sort_values(by="Credits", ascending=True)

    for _, player in remaining_players.iterrows():
        if len(selected_team) < 11:
            selected_team.append(player)

# Convert selected players into a DataFrame
df_selected_team = pd.DataFrame(selected_team)

# Display the final team
print(df_selected_team[["Player_Name", "Team", "Role", "Credits"]])


          Player_Name            Team  Role  Credits
157      Shubman Gill  Gujarat Titans   BAT      9.0
155      Shreyas Iyer    Punjab Kings   BAT      8.5
154      Shreyas Iyer    Punjab Kings   ALL      8.5
21     Arshdeep Singh    Punjab Kings   ALL      8.5
190  Yuzvendra Chahal    Punjab Kings   ALL      8.5
191  Yuzvendra Chahal    Punjab Kings  BOWL      8.5
22     Arshdeep Singh    Punjab Kings  BOWL      8.5
149    Shashank Singh    Punjab Kings   ALL      7.5
104     Nehal Wadhera    Punjab Kings   BAT      7.5
147     Shahrukh Khan  Gujarat Titans   BAT      7.5
103     Nehal Wadhera    Punjab Kings   ALL      7.5


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Load dataset
df = pd.read_csv("/content/cricket_data_with_credits.csv")

# Handle missing values (fill with median or zeros)
df.fillna(0, inplace=True)

# Encode categorical variables
encoder = LabelEncoder()
df["Player_Name"] = encoder.fit_transform(df["Player_Name"])
df["Role"] = encoder.fit_transform(df["Role"])
df["Team"] = encoder.fit_transform(df["Team"])

# Normalize numerical features
scaler = MinMaxScaler()
num_cols = [
    "Matches_Batted", "Not_Outs", "Runs_Scored", "Balls_Faced", "Batting_Average",
    "Batting_Strike_Rate", "Matches_Bowled", "Balls_Bowled", "Wickets_Taken",
    "Bowling_Average", "Economy_Rate", "Bowling_Strike_Rate", "Four_Wicket_Hauls",
    "Five_Wicket_Hauls", "Credits"
]
df[num_cols] = scaler.fit_transform(df[num_cols])

print(df.head())  # Check cleaned & normalized data


   Player_Name  Role  Team  Matches_Batted  Not_Outs  Runs_Scored  \
0            0     0     4        0.134409  0.111111     0.068485   
1            0     1     4        0.201613  0.250000     0.159545   
2            1     1     8        0.112903  0.083333     0.100000   
3            2     1     9        0.102151  0.027778     0.070000   
4            3     0     9        0.161290  0.069444     0.199091   

   Highest_Score  Batting_Average  Balls_Faced  Batting_Strike_Rate  ...  \
0      21.666667         0.193280     0.064634             0.620709  ...   
1       0.000000         0.479762     0.143902             0.787192  ...   
2      40.000000         0.290476     0.109756             0.640126  ...   
3      31.000000         0.219577     0.070732             0.589488  ...   
4      51.500000         0.312910     0.173577             0.732268  ...   

   Balls_Bowled  Runs_Conceded  Wickets_Taken  Best_Bowling_Match  \
0        0.0150      37.666667       0.014815              

In [ ]:
import pandas as pd

# Load dataset with encoding fixes
df = pd.read_csv("/content/cricket_data_with_credits.csv", encoding="utf-8")

# Ensure no spaces or hidden characters in column names
df.columns = df.columns.str.strip().str.replace("\ufeff", "")

# Rename common variations of column names
df.rename(columns={
    "Batting Avg": "Batting_Average",
    "Batting Average": "Batting_Average",
    " Bowling_Average": "Bowling_Average",  # Fix leading spaces
    "Bowling Avg": "Bowling_Average"
}, inplace=True)

# Debug: Print column names to verify corrections
print("✅ Updated Column Names:", df.columns.tolist())

# Check if required columns exist
required_columns = ["Batting_Average", "Bowling_Average"]
missing_columns = [col for col in required_columns if col not in df.columns]

if missing_columns:
    print(f"❌ Missing columns: {missing_columns}")
else:
    # Define a function to compute rolling averages safely
    def last_n_avg(series, n=5):
        return series.rolling(window=n, min_periods=1).mean()

    # Compute last 5-match batting average (Fix: Align index properly)
    df["Recent_Batting_Avg"] = df.groupby("Player_Name", group_keys=False)["Batting_Average"].apply(
        lambda x: last_n_avg(x, n=5)
    ).reset_index(level=0, drop=True)

    # Compute last 5-match bowling average (Fix: Align index properly)
    df["Recent_Bowling_Avg"] = df.groupby("Player_Name", group_keys=False)["Bowling_Average"].apply(
        lambda x: last_n_avg(x, n=5)
    ).reset_index(level=0, drop=True)

    print("✅ Successfully calculated Recent_Batting_Avg and Recent_Bowling_Avg!")

# Display the first few rows
print(df.head())

# Save the cleaned dataset
df.to_csv("cleaned_player_stats.csv", index=False)
print("✅ Cleaned dataset saved successfully!")


✅ Updated Column Names: ['Player_Name', 'Role', 'Team', 'Matches_Batted', 'Not_Outs', 'Runs_Scored', 'Highest_Score', 'Batting_Average', 'Balls_Faced', 'Batting_Strike_Rate', 'Centuries', 'Half_Centuries', 'Fours', 'Sixes', 'Catches_Taken', 'Stumpings', 'Matches_Bowled', 'Balls_Bowled', 'Runs_Conceded', 'Wickets_Taken', 'Best_Bowling_Match', 'Bowling_Average', 'Economy_Rate', 'Bowling_Strike_Rate', 'Four_Wicket_Hauls', 'Five_Wicket_Hauls', 'Credits']
✅ Successfully calculated Recent_Batting_Avg and Recent_Bowling_Avg!
        Player_Name Role                         Team  Matches_Batted  \
0       Abdul Samad  ALL         Lucknow Super Giants        8.333333   
1       Abdul Samad  BAT         Lucknow Super Giants       12.500000   
2  Abhinandan Singh  BAT  Royal Challengers Bengaluru        7.000000   
3   Abhinav Manohar  BAT          Sunrisers Hyderabad        6.333333   
4   Abhishek Sharma  ALL          Sunrisers Hyderabad       10.000000   

   Not_Outs  Runs_Scored  Highest_Sco

In [ ]:
# Calculate Fantasy Points
df["Fantasy_Points"] = (
    df["Runs_Scored"] * 1 +
    df["Wickets_Taken"] * 25 +
    df["Half_Centuries"] * 8 +
    df["Centuries"] * 16 +
    df["Four_Wicket_Hauls"] * 8 +
    df["Five_Wicket_Hauls"] * 16 +
    df.apply(lambda x: 6 if x["Economy_Rate"] < 6 else (4 if x["Economy_Rate"] < 7 else 0), axis=1) +
    df.apply(lambda x: 6 if x["Batting_Strike_Rate"] > 140 else (4 if x["Batting_Strike_Rate"] > 100 else 0), axis=1)
)

print("✅ Fantasy points calculated successfully!")


✅ Fantasy points calculated successfully!


In [ ]:
df

,Player_Name,Role,Team,Matches_Batted,Not_Outs,Runs_Scored,Highest_Score,Batting_Average,Balls_Faced,Batting_Strike_Rate,...,Best_Bowling_Match,Bowling_Average,Economy_Rate,Bowling_Strike_Rate,Four_Wicket_Hauls,Five_Wicket_Hauls,Credits,Recent_Batting_Avg,Recent_Bowling_Avg,Fantasy_Points
0,Abdul Samad,ALL,Lucknow Super Giants,8.333333,1.333333,75.333333,21.666667,12.176667,53.000000,118.493333,...,NaN,35.000000,10.236667,16.000000,0.000000,0.000000,6.5,12.176667,35.000000,96.000000
1,Abdul Samad,BAT,Lucknow Super Giants,12.500000,3.000000,175.500000,NaN,30.225000,118.000000,150.275000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,6.5,21.200833,17.500000,187.500000
2,Abhinandan Singh,BAT,Royal Challengers Bengaluru,7.000000,1.000000,110.000000,40.000000,18.300000,90.000000,122.200000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.0,18.300000,0.000000,120.000000
3,Abhinav Manohar,BAT,Sunrisers Hyderabad,6.333333,0.333333,77.000000,31.000000,13.833333,58.000000,112.533333,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,6.5,13.833333,0.000000,87.000000
4,Abhishek Sharma,ALL,Sunrisers Hyderabad,10.000000,0.833333,219.000000,51.500000,19.713333,142.333333,139.790000,...,NaN,28.333333,8.853333,19.000000,0.000000,0.000000,8.5,19.713333,28.333333,278.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,Yashasvi Jaiswal,ALL,Rajasthan Royals,10.000000,0.000000,258.000000,68.000000,25.800000,194.000000,132.990000,...,NaN,0.000000,36.000000,0.000000,0.000000,0.000000,9.0,25.800000,0.000000,278.000000
189,Yashasvi Jaiswal,BAT,Rajasthan Royals,10.500000,0.500000,337.250000,69.333333,29.345000,218.250000,139.657500,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,9.0,27.572500,0.000000,369.250000
190,Yuzvendra Chahal,ALL,Punjab Kings,14.714286,1.714286,5.285714,3.333333,1.904286,11.714286,60.051429,...,NaN,22.141429,7.602857,17.698571,0.285714,0.142857,8.5,1.904286,22.141429,484.857143
191,Yuzvendra Chahal,BOWL,Punjab Kings,11.400000,0.200000,0.000000,0.000000,0.000000,0.800000,0.000000,...,NaN,20.048000,8.300000,14.694000,0.800000,0.000000,8.5,0.952143,21.094714,366.400000


In [ ]:
print("Checking for NaN in Fantasy_Points:", df["Fantasy_Points"].isna().sum())
print("Checking for Inf in Fantasy_Points:", df["Fantasy_Points"].isin([float("inf"), float("-inf")]).sum())
df["Fantasy_Points"].fillna(df["Fantasy_Points"].mean(), inplace=True)
df["Fantasy_Points"].replace([float("inf"), float("-inf")], df["Fantasy_Points"].mean(), inplace=True)


Checking for NaN in Fantasy_Points: 0
Checking for Inf in Fantasy_Points: 0


<ipython-input-17-acf6cf8cf40c>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Fantasy_Points"].fillna(df["Fantasy_Points"].mean(), inplace=True)
<ipython-input-17-acf6cf8cf40c>:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, in

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Define features & target variable
features = ["Runs_Scored", "Wickets_Taken", "Batting_Strike_Rate", "Economy_Rate", "Recent_Batting_Avg", "Recent_Bowling_Avg", "Credits"]
X = df[features]
y = df["Fantasy_Points"]

# Split dataset into train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the XGBoost model
model = XGBRegressor(n_estimators=100, learning_rate=0.05)
model.fit(X_train, y_train)

# Save the trained model
import pickle
pickle.dump(model, open("fantasy_model.pkl", "wb"))

print("✅ Fantasy points prediction model trained & saved!")


✅ Fantasy points prediction model trained & saved!


In [ ]:
import pandas as pd
import pickle

# Load cleaned player dataset
df = pd.read_csv("cleaned_player_stats.csv")

# Load trained fantasy prediction model
model = pickle.load(open("fantasy_model.pkl", "rb"))


In [ ]:
import pandas as pd
import pickle

# ✅ Load cleaned player dataset
df = pd.read_csv("cleaned_player_stats.csv")

# ✅ Load trained fantasy prediction model
model = pickle.load(open("fantasy_model.pkl", "rb"))

# ✅ Standardize Role Names
# ✅ Define a role mapping dictionary
role_mapping = {
    "Batsman": "BAT",
    "Allrounder": "ALL",
    "Wicketkeeper": "WK",
     "WK-Batter":"WK",
    "Wicket Keeper/Batsman": "WK",
    "Bowler": "BOWL"
}

# ✅ Apply role mapping to fix roles
df["Role"] = df["Role"].replace(role_mapping)

print("✅ Role names standardized successfully!")
print("Updated unique roles:", df["Role"].unique())  # Check if mapping worked


# ✅ Function to select fantasy team
def select_fantasy_team(team1, team2):
    match_players = df[(df["Team"] == team1) | (df["Team"] == team2)].copy()

    print(f"Players found for {team1} vs {team2}: {len(match_players)}")
    if match_players.empty:
        print("❌ No players found for the selected teams. Check team names!")
        return None

    # ✅ Predict fantasy points
    features = ["Runs_Scored", "Wickets_Taken", "Batting_Strike_Rate", "Economy_Rate", "Recent_Batting_Avg", "Recent_Bowling_Avg", "Credits"]
    X_match = match_players[features]
    match_players["Predicted_Fantasy_Points"] = model.predict(X_match)

    # ✅ Sort players by predicted fantasy points
    match_players = match_players.sort_values(by="Predicted_Fantasy_Points", ascending=False)

    # ✅ Fantasy selection rules
    selected_team = []
    budget = 100
    team_counts = {"WK": 0, "BAT": 0, "ALL": 0, "BOWL": 0}

    for _, player in match_players.iterrows():
        role = player["Role"]

        # ✅ Check if role exists in our dictionary
        if role not in team_counts:
            print(f"❌ Unexpected role: {role}. Skipping player: {player['Player_Name']}")
            continue

        # ✅ Ensure team balance
        if (role == "WK" and team_counts["WK"] >= 2) or \
           (role == "BAT" and team_counts["BAT"] >= 3) or \
           (role == "ALL" and team_counts["ALL"] >= 4) or \
           (role == "BOWL" and team_counts["BOWL"] >= 3):
            continue

        # ✅ Check budget
        if budget - player["Credits"] >= 0:
            selected_team.append(player["Player_Name"])
            budget -= player["Credits"]
            team_counts[role] += 1  # Update role count

        if len(selected_team) == 11:
            break

    # ✅ Handle case where fewer than 11 players were selected
    if len(selected_team) < 11:
        print(f"❌ Only {len(selected_team)} players selected, cannot form a full fantasy team.")
        return None

    # ✅ Assign Captain & Vice-Captain
    selected_team_df = match_players[match_players["Player_Name"].isin(selected_team)]
    captain = selected_team_df.iloc[0]["Player_Name"]
    vice_captain = selected_team_df.iloc[1]["Player_Name"]

    # ✅ Add C/VC column
    selected_team_df["C/VC"] = selected_team_df["Player_Name"].apply(lambda x: "C" if x == captain else ("VC" if x == vice_captain else "NA"))


    return selected_team_df

# ✅ Example Usage: Select fantasy team for "Gujarat Titans" vs "Mumbai Indians"
fantasy_team = select_fantasy_team("Gujarat Titans", "Punjab Kings")
print(fantasy_team)


✅ Role names standardized successfully!
Updated unique roles: ['ALL' 'BAT' 'BOWL' 'WK']
Players found for Gujarat Titans vs Punjab Kings: 41
            Player_Name  Role            Team  Matches_Batted  Not_Outs  \
185     Xavier Bartlett  BOWL    Punjab Kings        8.000000  4.000000   
134          Robin Minz    WK  Gujarat Titans       30.000000  7.000000   
101        Musheer Khan   ALL    Punjab Kings       18.000000  5.000000   
157        Shubman Gill   BAT  Gujarat Titans       14.714286  2.142857   
77   Kulwant Khejroliya  BOWL  Gujarat Titans        3.333333  1.333333   
190    Yuzvendra Chahal   ALL    Punjab Kings       14.714286  1.714286   
112       Priyansh Arya   BAT    Punjab Kings       25.000000  6.000000   
154        Shreyas Iyer   ALL    Punjab Kings       14.000000  1.000000   
21       Arshdeep Singh   ALL    Punjab Kings       13.500000  1.750000   
191    Yuzvendra Chahal  BOWL    Punjab Kings       11.400000  0.200000   
155        Shreyas Iyer   BAT    P

<ipython-input-20-bac10776d715>:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_team_df["C/VC"] = selected_team_df["Player_Name"].apply(lambda x: "C" if x == captain else ("VC" if x == vice_captain else "NA"))


In [ ]:
import pandas as pd
import pickle

# ✅ Load cleaned player dataset
df = pd.read_csv("cleaned_player_stats.csv")

# ✅ Load trained fantasy prediction model
model = pickle.load(open("fantasy_model.pkl", "rb"))

# ✅ Standardize Role Names
role_mapping = {
    "Batsman": "BAT",
    "Allrounder": "ALL",
    "Wicketkeeper": "WK",
    "WK-Batter": "WK",
    "Wicket Keeper/Batsman": "WK",
    "Bowler": "BOWL"
}
df["Role"] = df["Role"].replace(role_mapping)

print("✅ Role names standardized successfully!")
print("Updated unique roles:", df["Role"].unique())

# ✅ Function to select fantasy team
def select_fantasy_team(team1, team2):
    match_players = df[(df["Team"] == team1) | (df["Team"] == team2)].copy()

    print(f"Players found for {team1} vs {team2}: {len(match_players)}")
    if match_players.empty:
        print("❌ No players found for the selected teams. Check team names!")
        return None

    # ✅ Predict fantasy points
    features = ["Runs_Scored", "Wickets_Taken", "Batting_Strike_Rate", "Economy_Rate", "Recent_Batting_Avg", "Recent_Bowling_Avg", "Credits"]
    X_match = match_players[features]
    match_players["Predicted_Fantasy_Points"] = model.predict(X_match)

    # ✅ Sort players by predicted fantasy points and then credits (low to high)
    match_players = match_players.sort_values(by=["Predicted_Fantasy_Points", "Credits"], ascending=[False, True])

    # ✅ Fantasy selection rules
    selected_team = []
    budget = 100
    max_role_limits = {"WK": 2, "BAT": 5, "ALL": 3, "BOWL": 5}
    team_counts = {"WK": 0, "BAT": 0, "ALL": 0, "BOWL": 0}

    for _, player in match_players.iterrows():
        role = player["Role"]

        # ✅ Check if role exists in our dictionary
        if role not in team_counts:
            print(f"❌ Unexpected role: {role}. Skipping player: {player['Player_Name']}")
            continue

        # ✅ Allow more flexibility in role selection
        if team_counts[role] >= max_role_limits[role]:
            continue

        # ✅ Ensure budget is properly utilized
        if budget - player["Credits"] >= 0:
            selected_team.append(player["Player_Name"])
            budget -= player["Credits"]
            team_counts[role] += 1

        if len(selected_team) == 15:
            break

    # ✅ If fewer than 11 players, allow best remaining players to be added
    if len(selected_team) < 11:
        print("⚠ Warning: Not enough players selected. Adding best remaining players.")
        remaining_players = match_players[~match_players["Player_Name"].isin(selected_team)]

        for _, player in remaining_players.iterrows():
            if len(selected_team) < 11 and budget - player["Credits"] >= 0:
                selected_team.append(player["Player_Name"])
                budget -= player["Credits"]

    # ✅ Handle case where still fewer than 11 players
    if len(selected_team) < 11:
        print(f"❌ Only {len(selected_team)} players selected, cannot form a full fantasy team.")
        return None

    # ✅ Assign Captain & Vice-Captain
    selected_team_df = match_players[match_players["Player_Name"].isin(selected_team)]
    captain = selected_team_df.iloc[0]["Player_Name"]
    vice_captain = selected_team_df.iloc[1]["Player_Name"]

    # ✅ Add C/VC column
    selected_team_df["C/VC"] = selected_team_df["Player_Name"].apply(lambda x: "C" if x == captain else ("VC" if x == vice_captain else "NA"))

    # ✅ Save final team
    selected_team_df.to_csv(f"fantasy_team_{team1}_vs_{team2}.csv", index=False)

    print(f"✅ Fantasy team selected for {team1} vs {team2}!")

    # ✅ Print selected player names separately
    print("\n🔹 **Selected Fantasy Team:**")
    for i, player in enumerate(selected_team, start=1):
        print(f"{i}. {player}")

    print(f"\n🔹 **Captain:** {captain}")
    print(f"🔹 **Vice-Captain:** {vice_captain}")

    return selected_team_df

# ✅ Example Usage: Select fantasy team for "Gujarat Titans" vs "Mumbai Indians"
fantasy_team = select_fantasy_team("Sunrisers Hyderabad", "Lucknow Super Giants")
print(fantasy_team)

#


✅ Role names standardized successfully!
Updated unique roles: ['ALL' 'BAT' 'BOWL' 'WK']
Players found for Sunrisers Hyderabad vs Lucknow Super Giants: 38
✅ Fantasy team selected for Sunrisers Hyderabad vs Lucknow Super Giants!

🔹 **Selected Fantasy Team:**
1. Matthew Breetzke
2. Aryan Juyal
3. Prince Yadav
4. Rishabh Pant
5. Zeeshan Ansari
6. Harshal Patel
7. Avesh Khan
8. Ravi Bishnoi
9. Ishan Kishan
10. Rishabh Pant
11. Atharva Taide
12. Rahul Chahar
13. Abdul Samad
14. Mayank Yadav

🔹 **Captain:** Matthew Breetzke
🔹 **Vice-Captain:** Aryan Juyal
          Player_Name  Role                  Team  Matches_Batted  Not_Outs  \
89   Matthew Breetzke   BAT  Lucknow Super Giants       45.000000  8.000000   
24        Aryan Juyal    WK  Lucknow Super Giants       40.000000  8.000000   
111      Prince Yadav   ALL  Lucknow Super Giants       22.000000  6.000000   
131      Rishabh Pant    WK  Lucknow Super Giants       14.500000  2.000000   
192    Zeeshan Ansari  BOWL   Sunrisers Hyderabad 

<ipython-input-23-2eef5f6c6bbe>:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_team_df["C/VC"] = selected_team_df["Player_Name"].apply(lambda x: "C" if x == captain else ("VC" if x == vice_captain else "NA"))


In [ ]:
import pandas as pd
import pickle

# ✅ Load cleaned player dataset
df = pd.read_csv("cleaned_player_stats.csv")

# ✅ Load trained fantasy prediction model
model = pickle.load(open("fantasy_model.pkl", "rb"))

# ✅ Standardize Role Names
role_mapping = {
    "Batsman": "BAT",
    "Allrounder": "ALL",
    "Wicketkeeper": "WK",
    "WK-Batter": "WK",
    "Wicket Keeper/Batsman": "WK",
    "Bowler": "BOWL"
}
df["Role"] = df["Role"].replace(role_mapping)

print("✅ Role names standardized successfully!")

# ✅ Function to select fantasy team
def select_fantasy_team(team1, team2):
    match_players = df[(df["Team"] == team1) | (df["Team"] == team2)].copy()

    if match_players.empty:
        print(f"❌ No players found for {team1} vs {team2}. Check team names!")
        return None

    print(f"✅ Players found for {team1} vs {team2}: {len(match_players)}")

    # ✅ Check required features exist
    required_features = ["Runs_Scored", "Wickets_Taken", "Batting_Strike_Rate", "Economy_Rate",
                         "Recent_Batting_Avg", "Recent_Bowling_Avg", "Credits"]

    missing_features = [feat for feat in required_features if feat not in match_players.columns]
    if missing_features:
        print(f"❌ Missing features in dataset: {missing_features}")
        return None

    # ✅ Predict fantasy points
    X_match = match_players[required_features]
    match_players["Predicted_Fantasy_Points"] = model.predict(X_match)

    # ✅ Sort players by fantasy points and credits
    match_players = match_players.sort_values(by=["Predicted_Fantasy_Points", "Credits"], ascending=[False, True])

    # ✅ Fantasy selection rules
    selected_team = []
    selected_players = set()
    budget = 100
    max_role_limits = {"WK": 2, "BAT": 5, "ALL": 3, "BOWL": 5}
    team_counts = {role: 0 for role in max_role_limits}

    for _, player in match_players.iterrows():
        role, player_name, credits = player["Role"], player["Player_Name"], player["Credits"]

        # ✅ Ensure unique players & role limits
        if player_name in selected_players or team_counts[role] >= max_role_limits[role]:
            continue

        # ✅ Ensure budget constraint
        if budget - credits >= 0:
            selected_team.append(player)
            selected_players.add(player_name)
            budget -= credits
            team_counts[role] += 1

        if len(selected_team) == 11:
            break

    # ✅ Fill remaining slots with best available players
    if len(selected_team) < 11:
        print("⚠ Warning: Not enough players selected. Adding best remaining players.")
        remaining_players = match_players[~match_players["Player_Name"].isin(selected_players)]

        for _, player in remaining_players.iterrows():
            if len(selected_team) < 11 and budget - player["Credits"] >= 0:
                selected_team.append(player)
                selected_players.add(player["Player_Name"])
                budget -= player["Credits"]

    if len(selected_team) < 11:
        print(f"❌ Only {len(selected_team)} players selected, cannot form a full fantasy team.")
        return None

    # ✅ Convert selected team to DataFrame
    selected_team_df = pd.DataFrame(selected_team)

    # ✅ Assign Captain & Vice-Captain
    selected_team_df["C/VC"] = "NA"
    selected_team_df.loc[selected_team_df.index[0], "C/VC"] = "C"
    selected_team_df.loc[selected_team_df.index[1], "C/VC"] = "VC"

    # ✅ Keep only required columns
    selected_team_df = selected_team_df[["Player_Name", "Credits", "Predicted_Fantasy_Points", "Role", "C/VC"]]

    # ✅ Save final team
    csv_filename = f"fantasy_team_{team1}_vs_{team2}.csv"
    selected_team_df.to_csv(csv_filename, index=False)

    print(f"✅ Fantasy team selected for {team1} vs {team2} and saved as {csv_filename}!")

    # ✅ Print selected player names
    print("\n🔹 **Selected Fantasy Team:**")
    for i, row in selected_team_df.iterrows():
        print(f"{i+1}. {row['Player_Name']} ({row['Role']}) - {row['Credits']} credits, {row['Predicted_Fantasy_Points']:.2f} points")

    print(f"\n🔹 **Captain:** {selected_team_df.iloc[0]['Player_Name']}")
    print(f"🔹 **Vice-Captain:** {selected_team_df.iloc[1]['Player_Name']}")

    return selected_team_df, csv_filename

# ✅ Example Usage
team1, team2 = "Sunrises Hydrabad", "Lucknow Super Giants"
fantasy_team_df, csv_filename = select_fantasy_team(team1, team2)

# ✅ Download File (for Google Colab)
if fantasy_team_df is not None:
    from google.colab import files
    files.download(csv_filename)


✅ Role names standardized successfully!
✅ Players found for Sunrises Hydrabad vs Lucknow Super Giants: 21
✅ Fantasy team selected for Sunrises Hydrabad vs Lucknow Super Giants and saved as fantasy_team_Sunrises Hydrabad_vs_Lucknow Super Giants.csv!

🔹 **Selected Fantasy Team:**
90. Matthew Breetzke (BAT) - 6.5 credits, 1043.43 points
25. Aryan Juyal (WK) - 5.5 credits, 670.11 points
112. Prince Yadav (ALL) - 5.5 credits, 544.41 points
132. Rishabh Pant (WK) - 9.0 credits, 490.76 points
31. Avesh Khan (ALL) - 8.0 credits, 378.88 points
127. Ravi Bishnoi (BOWL) - 8.0 credits, 323.62 points
97. Mohsin Khan (ALL) - 7.0 credits, 323.33 points
2. Abdul Samad (BAT) - 6.5 credits, 190.71 points
93. Mayank Yadav (BOWL) - 7.5 credits, 173.70 points
44. Digvesh Singh (BAT) - 5.0 credits, 136.88 points
12. Akash Deep (BOWL) - 7.0 credits, 125.18 points

🔹 **Captain:** Matthew Breetzke
🔹 **Vice-Captain:** Aryan Juyal


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

final work
**bold text**

In [ ]:


import pandas as pd
d1= pd.read_csv("/content/drive/MyDrive/Gamethon/SquadPlayerNames_IndianT20League - SquadData_AllTeams.csv")
d1.rename(columns={"Player Name": "Player_Name"}, inplace=True)
d1.head()


,Credits,Player Type,Player_Name,Team
0,5.0,WK,Vansh Bedi,CHE
1,5.5,BAT,Andre Siddharth,CHE
2,5.5,ALL,Ramakrishna Ghosh,CHE
3,5.0,BAT,Shaik Rasheed,CHE
4,6.0,BOWL,Gurjapneet Singh,CHE


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['Credits'].plot(kind='hist', bins=20, title='Credits')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_2.groupby('Player Type').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_3.groupby('Player_Name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_13['Player_Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_13, x='index', y='Player_Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_14['Player Type'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_14, x='Credits', y='Player Type', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_15['Player_Name'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_15, x='Credits', y='Player_Name', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

✅ Combined dataset saved successfully!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Load the CSV file
d2 = pd.read_excel("/Player_Stats_Final_Complete.xlsx")


# Function to search for a player's name and retrieve their data
def search_player(player_name):
    result = d2[d2["Player_Name"].str.contains(player_name, case=False, na=False)]
    if not result.empty:
        return result
    else:
        return "Player not found."

# Example usage
player_name = input("Enter player name: ")
print(search_player(player_name))


Enter player name: Andre Siddharth
Player not found.


In [ ]:


import pandas as pd
df = pd.read_csv("/content/Players_stats.csv")
df


,Credits_x,Player Type,Player_Name,Team,Matches_Batted,Not_Outs,Runs_Scored,Highest_Score,Batting_Average,Balls_Faced,...,Wickets_Taken,Best_Bowling_Match,Bowling_Average,Economy_Rate,Bowling_Strike_Rate,Four_Wicket_Hauls,Five_Wicket_Hauls,Credits_y,Recent_Batting_Avg,Recent_Bowling_Avg
0,5.5,BAT,Andre Siddharth,CHE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6.0,BOWL,Anshul Kamboj,CHE,3.0,1.0,2.0,NaN,0.000,2.0,...,2.0,NaN,57.000,11.400,30.0,0.0,0.0,6.0,0.000,57.000
2,7.0,BAT,Deepak Hooda,CHE,11.8,1.6,146.5,42.571429,26.044,113.1,...,1.0,NaN,31.433,8.854,21.3,0.0,0.0,7.0,26.044,31.433
3,8.0,WK,Devon Conway,CHE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,BOWL,Gurjapneet Singh,CHE,5.0,3.0,22.0,NaN,11.000,30.0,...,29.0,NaN,29.100,7.500,23.2,0.0,0.0,6.0,11.000,29.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,6.5,BAT,Sachin Baby,SRH,11.0,1.0,119.0,33.000000,23.800,79.0,...,2.0,NaN,4.000,4.800,5.0,0.0,0.0,6.5,23.800,4.000
228,6.0,BOWL,Simarjeet- Singh,SRH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
229,9.0,BAT,Travis Head,SRH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
230,7.0,ALL,Wiaan Mulder,SRH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

def filter_players_with_nan(df,nan_threshold=2):
    # Load the dataset
    df

    # Standardize column names by stripping whitespace
    df.columns = df.columns.str.strip()

    # Filter rows where the number of NaN values is greater than the threshold
    filtered_players = df[df.isna().sum(axis=1) > nan_threshold]

    return filtered_players

nan_threshold = 2  # Set the threshold for missing values
filtered_players = filter_players_with_nan(df, nan_threshold)
print(filtered_players)

     Credits_x Player Type          Player_Name Team  Matches_Batted  \
0          5.5         BAT      Andre Siddharth  CHE             NaN   
3          8.0          WK         Devon Conway  CHE             NaN   
7          7.5        BOWL        Khaleel Ahmed  CHE             NaN   
9          8.5        BOWL  Matheesha Pathirana  CHE             NaN   
11         7.0        BOWL         Nathan Ellis  CHE             NaN   
..         ...         ...                  ...  ...             ...   
224        8.5        BOWL       Mohammed Shami  SRH             NaN   
225        8.5        BOWL          Pat Cummins  SRH             NaN   
228        6.0        BOWL     Simarjeet- Singh  SRH             NaN   
229        9.0         BAT          Travis Head  SRH             NaN   
230        7.0         ALL         Wiaan Mulder  SRH             NaN   

     Not_Outs  Runs_Scored  Highest_Score  Batting_Average  Balls_Faced  ...  \
0         NaN          NaN            NaN              

In [ ]:
# prompt: download filtered_players csv

import pandas as pd
from google.colab import files

filtered_players.to_csv("filtered_players.csv", index=False)

# Download the CSV file
files.download("filtered_players.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Load datasets
d1 = pd.read_csv("/content/Players_stats.csv")  # Player stats dataset
d2 = pd.read_excel("/content/missed data.xlsx")  # Another dataset to combine with


# Filter d1 to include only players present in d2
filtered_d1 = d1[d1["Player_Name"].isin(d2["Player_Name"])]

# Combine with d2
combined_df = pd.concat([d2, filtered_d1], ignore_index=True)


# Display the first few rows
combined_df



,Credits_x,Player Type,Player_Name,Team,Matches_Batted,Not_Outs,Runs_Scored,Highest_Score,Batting_Average,Balls_Faced,...,Wickets_Taken,Best_Bowling_Match,Bowling_Average,Economy_Rate,Bowling_Strike_Rate,Four_Wicket_Hauls,Five_Wicket_Hauls,Credits_y,Recent_Batting_Avg,Recent_Bowling_Avg
0,5.5,BAT,Andre Siddharth,CHE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.0,WK,Devon Conway,CHE,23.0,3.0,924.0,92*,48.63,654.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,7.5,BOWL,Khaleel Ahmed,CHE,60.0,0.0,1.0,1,0.25,7.0,...,80.0,2021-03-01 00:00:00,24.7,8.8,16.85,0.0,0.0,NaN,NaN,NaN
3,8.5,BOWL,Matheesha Pathirana,CHE,22.0,0.0,0.0,0,NaN,NaN,...,38.0,4\28,17.26,7.89,13.13,1.0,0.0,NaN,NaN,NaN
4,7.0,BOWL,Nathan Ellis,CHE,17.0,1.0,19.0,12,4.75,25.0,...,19.0,4\30,29.16,8.66,20.21,1.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,8.5,BOWL,Mohammed Shami,SRH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,8.5,BOWL,Pat Cummins,SRH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,6.0,BOWL,Simarjeet- Singh,SRH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,9.0,BAT,Travis Head,SRH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# prompt: print d2.cplumn names

d2.columns


Index(['Credits_x', 'Player Type', 'Player_Name', 'Team', 'Matches_Batted',
       'Not_Outs', 'Runs_Scored', 'Highest_Score', 'Batting_Average',
       'Balls_Faced', 'Batting_Strike_Rate', 'Centuries', 'Half_Centuries',
       'Fours', 'Sixes', 'Catches_Taken', 'Stumpings', 'Matches_Bowled',
       'Balls_Bowled', 'Runs_Conceded', 'Wickets_Taken', 'Best_Bowling_Match',
       'Bowling_Average', 'Economy_Rate', 'Bowling_Strike_Rate',
       'Four_Wicket_Hauls', 'Five_Wicket_Hauls', 'Credits_y',
       'Recent_Batting_Avg', 'Recent_Bowling_Avg'],
      dtype='object')

In [ ]:
# prompt: /Player_Stats_Final_Complete.xlsx load it

import pandas as pd
from google.colab import files

# Load the Excel file
df = pd.read_excel("/content/Player_Stats_Final_Complete.xlsx")

# Display the first few rows of the DataFrame
df


,Sno,Player_Name,India/Overseas,Role,Team,Auction Type,Auction Price,Year,Matches_Batted,Not_Outs,...,Matches_Bowled,Balls_Bowled,Runs_Conceded,Wickets_Taken,Best_Bowling_Match,Bowling_Average,Economy_Rate,Bowling_Strike_Rate,Four_Wicket_Hauls,Five_Wicket_Hauls
0,1,Shaik Rasheed,India,Batter,NaN,Uncapped,0,2025,28,5,...,2,24,45,1,1/18,45.0,11.2,24.0,0,0.0
1,2,Gurjapneet Singh,India,Bowler,NaN,Uncapped,0,2025,5,3,...,32,672,845,29,3/22,29.1,7.5,23.2,0,0.0
2,3,Ramakrishna Ghosh,India,All-rounder,NaN,Uncapped,0,2025,18,4,...,15,300,390,12,2/15,32.5,7.8,25.0,0,0.0
3,4,Jamie Overton,Overseas (ENG),All-rounder,NaN,Overseas,0,2025,62,12,...,58,1200,1620,45,4/20,36.0,8.1,26.7,2,0.0
4,5,Vansh Bedi,India,Batter,NaN,Uncapped,0,2025,15,2,...,0,0,0,0,0.0,0.0,0.0,0.0,0,NaN
5,6,Vipraj Nigam,India,Bowler,NaN,Uncapped,0,2025,8,5,...,25,540,680,22,3/25,30.9,7.6,24.5,0,0.0
6,7,Ajay Mandal,India,All-rounder,NaN,Uncapped,0,2025,20,6,...,18,360,480,14,3/30,34.3,8.0,25.7,0,0.0
7,8,Madhav Tiwari,India,Batter,NaN,Uncapped,0,2025,12,1,...,0,0,0,0,0.0,0.0,0.0,0.0,0,NaN
8,9,Tripurana Vijay,India,Batter,NaN,Uncapped,0,2025,10,2,...,0,0,0,0,0.0,0.0,0.0,0.0,0,NaN
9,10,Nishant Sindhu,India,All-rounder,CSK,Uncapped,0,2025,25,8,...,22,420,550,18,3/15,30.6,7.9,23.3,0,0.0


In [ ]:
# prompt: /content/cric.csv loacdit'

import pandas as pd
from google.colab import files

# Load the CSV file
df = pd.read_csv('/content/cric.csv')

# Display the first few rows of the DataFrame
df


,Credits_x,Player Type,Player_Name,Team,Matches_Batted,Not_Outs,Runs_Scored,Highest_Score,Batting_Average,Balls_Faced,...,Wickets_Taken,Best_Bowling_Match,Bowling_Average,Economy_Rate,Bowling_Strike_Rate,Four_Wicket_Hauls,Five_Wicket_Hauls,Credits_y,Recent_Batting_Avg,Recent_Bowling_Avg
0,9.5,Batsman,Devon Conway,CSK,22,3,924,92*,48.63,654,...,0,-,0.00,0.00,0.00,0,0,9.5,48.63,0.00
1,8.0,Bowler,Khaleel Ahmed,DC,0,0,0,0,0.00,0,...,32,4/18,25.75,7.05,21.38,3,1,8.0,0.00,25.75
2,8.5,Bowler,Matheesha Pathirana,CSK,0,0,0,0,0.00,0,...,28,3/24,26.78,6.45,22.14,2,0,8.5,0.00,26.78
3,7.5,Bowler,Nathan Ellis,KXIP,0,0,0,0,0.00,0,...,40,5/34,20.50,7.20,18.50,1,1,7.5,0.00,20.50
4,7.0,Bowler,Noor Ahmad,GT,0,0,0,0,0.00,0,...,25,4/27,23.50,7.90,22.50,1,0,7.0,0.00,23.50
5,8.0,All-rounder,Rachin Ravindra,CSK,15,4,520,103*,47.27,380,...,15,2/25,17.33,6.50,20.00,0,0,8.0,47.27,17.33


In [ ]:
import pandas as pd

# Load d1 and d2 (Assuming they are in CSV format)
d2 = pd.read_excel("/content/missed data.xlsx")  # Replace with actual path
d1 = pd.read_csv("/content/Players_stats.csv")  # Replace with actual path

# Merge d1 and d2 on 'Player_Name' (Assuming d1 has additional stats)
combined_df = pd.merge(d2, d1, on="Player_Name", how="left")  # Left join to keep all d2 players

# Save the combined data
combined_df



,Credits_x_x,Player Type_x,Player_Name,Team_x,Matches_Batted_x,Not_Outs_x,Runs_Scored_x,Highest_Score_x,Batting_Average_x,Balls_Faced_x,...,Wickets_Taken_y,Best_Bowling_Match_y,Bowling_Average_y,Economy_Rate_y,Bowling_Strike_Rate_y,Four_Wicket_Hauls_y,Five_Wicket_Hauls_y,Credits_y_y,Recent_Batting_Avg_y,Recent_Bowling_Avg_y
0,5.5,BAT,Andre Siddharth,CHE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8.0,WK,Devon Conway,CHE,23.0,3.0,924.0,92*,48.63,654.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7.5,BOWL,Khaleel Ahmed,CHE,60.0,0.0,1.0,1,0.25,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8.5,BOWL,Matheesha Pathirana,CHE,22.0,0.0,0.0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.0,BOWL,Nathan Ellis,CHE,17.0,1.0,19.0,12,4.75,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,8.5,BOWL,Mohammed Shami,SRH,113.0,13.0,76.0,21,5.43,83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,8.5,BOWL,Pat Cummins,SRH,61.0,16.0,535.0,66*,19.11,356.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,6.0,BOWL,Simarjeet- Singh,SRH,12.0,3.0,10.0,3*,10,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,9.0,BAT,Travis Head,SRH,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd
from google.colab import files

# Load the datasets
d1 = pd.read_csv("/content/Players_stats.csv")
d2 = pd.read_excel("/content/missed data.xlsx")

# Merge d1 with d2 based on 'Player' column, filling missing stats
updated_d1 = d1.set_index('Player_Name').combine_first(d2.set_index('Player_Name')).reset_index()

# Save the updated dataset
updated_file_path = "Updated_d1.csv"
updated_d1.to_csv(updated_file_path, index=False)

# Download the updated file
files.download(updated_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import files


# Load the dataset
d1 = pd.read_csv("/content/Final_players_stats.csv")

# Fill NaN values with 0
d1_filled = d1.fillna(0)

# Save the updated dataset
updated_file_path = "Final_cricket_data.csv"
d1_filled.to_csv(updated_file_path, index=False)

# Download the updated file
files.download(updated_file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>